In [1]:
import os
import sys
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from glob import glob
from tqdm.notebook import tqdm

import xfeat
from scipy.stats import skew, kurtosis
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgbm

def RMSE(y_true, y_pred):
    return np.sqrt(np.mean(np.square(y_true-y_pred)))

In [2]:
def read_data(path: Path):
    train_weather = np.load(path / 'Training/inputs_weather_train.npy')
    train_other = np.load(path / 'Training/inputs_others_train.npy')
    train_y = np.load(path / 'Training/yield_train.npy')

    test_weather = np.load(path / 'Test Inputs/inputs_weather_test.npy')
    test_other = np.load(path / 'Test Inputs/inputs_others_test.npy')

    clusters = np.load(path / 'clusterID_genotype.npy')
    clusters = {i+1: c for i, c in enumerate(clusters)}
    
    weather_cols = ['ADNI', 'AP', 'ARH', 'MDNI', 'MaxSur', 'Minsur', 'AvgSur']
    other_cols = ['MG', 'ID', 'State', 'Year', 'Location']

    train = pd.DataFrame(train_other, columns=other_cols)
    test = pd.DataFrame(test_other, columns=other_cols)
        
    other_cols.remove('State')
    train[other_cols] = train[other_cols].astype(float).astype(int)
    test[other_cols] = test[other_cols].astype(float).astype(int)
    
    train['State'] = train['State'].map(lambda x: x.split('"')[1])
    test['State'] = test['State'].map(lambda x: x.split('"')[1])
    state2num = {c: i for i, c in enumerate(train['State'].unique())}
    train['label_State'] = train['State'].map(state2num)
    test['label_State'] = test['State'].map(state2num)
    
    train['cluster'] = train['ID'].map(clusters)
    test['cluster'] = test['ID'].map(clusters)
    
    other_cols.append('label_State')
    other_cols.append('cluster')
    
    tmp_cols = ['MG', 'ID', 'Year', 'Location', 'label_State', 'cluster']
    train[tmp_cols] = train[tmp_cols].astype(str)
    test[tmp_cols] = test[tmp_cols].astype(str)
    
    for i in [2, 3, 4, 5, 6]:
        tmp_train = xfeat.ConcatCombination(r=i).fit_transform(train[tmp_cols])
        tmp_test = xfeat.ConcatCombination(r=i).fit_transform(test[tmp_cols])
        new_cols = [c for c in tmp_train.columns if not c in tmp_cols]
        other_cols = other_cols + new_cols
        tmp_train = tmp_train[new_cols]
        tmp_test = tmp_test[new_cols]
        tmp_train = tmp_train.astype(int)
        tmp_test = tmp_test.astype(int)
        train = pd.concat([train, tmp_train], axis=1)
        test = pd.concat([test, tmp_test], axis=1)
        
    train[tmp_cols] = train[tmp_cols].astype(int)
    test[tmp_cols] = test[tmp_cols].astype(int)
    
    feature_cols = other_cols.copy()
    for col in other_cols:
        _count = train[col].value_counts()
        train[f'count_{col}'] = train[col].map(_count)
        test[f'count_{col}'] = test[col].map(_count)
        feature_cols.append(f'count_{col}')
        
    train['target'] = train_y
    
    for i, col in enumerate(weather_cols):
        tmp_columns = [f'{col}_{date+1}' for date in range(214)]
        tmp_train = pd.DataFrame(train_weather[:,:,i], columns=tmp_columns)
        train = pd.concat([train, tmp_train], axis=1)
        tmp_test = pd.DataFrame(test_weather[:,:,i], columns=tmp_columns)
        test = pd.concat([test, tmp_test], axis=1)
    
    return train, test, clusters, feature_cols

def get_stratifiedkfold(train, n_splits, seed):
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    fold_series = []
    for fold, (idx_train, idx_valid) in enumerate(kf.split(train, train['ID'])):
        fold_series.append(pd.Series(fold, index=idx_valid))
    fold_series = pd.concat(fold_series).sort_index()
    return fold_series

def processing_base(input_df: pd.DataFrame, feature_cols: list):
    output_df = input_df[feature_cols].copy()
    return output_df

def processing_statistics(input_df: pd.DataFrame):
    output_df = pd.DataFrame()
    weather_cols = ['ADNI', 'AP', 'ARH', 'MDNI', 'MaxSur', 'Minsur', 'AvgSur']
    for col in weather_cols:
        feature_columns = [c for c in input_df.columns if col in c]
        arr = input_df[feature_columns].values
        for func in [np.median, np.mean, np.min, np.max, np.std, skew]:
            output_df[col+'_'+func.__name__] = func(arr, axis=1)
    return output_df

def processing_statistics_diff(input_df: pd.DataFrame):
    output_df = pd.DataFrame()
    weather_cols = ['ADNI', 'AP', 'ARH', 'MDNI', 'MaxSur', 'Minsur', 'AvgSur']
    for col in weather_cols:
        feature_columns = [c for c in input_df.columns if col in c]
        arr = input_df[feature_columns].values
        for func in [np.median, np.mean, np.min, np.max, np.std, skew]:
            output_df[col+'_diff_'+func.__name__] = func(
                arr[:, 1:214] - arr[:, 0:213], axis=1)
    return output_df

def processing_targetencoding(train: pd.DataFrame, test: pd.DataFrame, col: str):
    fold_unique = sorted(folds.unique())
    for fold in fold_unique:
        idx_train = (folds!=fold)
        idx_valid = (folds==fold)
        group = train[idx_train].groupby(col)['target'].mean().to_dict()
        train.loc[idx_valid, col+'_target'] = train.loc[idx_valid, col].map(group)
    group = train.groupby(col)['target'].mean().to_dict()
    test[col+'_target'] = test[col].map(group)
    return train, test

def fit_lightgbm(X, y, params, folds, seed, add_suffix=''):
    oof_pred = np.zeros(len(y), dtype=np.float64)
    
    fold_unique = sorted(folds.unique())
    for fold in fold_unique:
        idx_train = (folds!=fold)
        idx_valid = (folds==fold)
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]
        
        lgbm_train = lgbm.Dataset(x_train, y_train)
        lgbm_valid = lgbm.Dataset(x_valid, y_valid, reference=lgbm_train)
        
        model = lgbm.train(
            params=params,
            train_set=lgbm_train,
            valid_sets=[lgbm_train, lgbm_valid],
            num_boost_round=10000,
            verbose_eval=100,
        )
        
        pickle.dump(model, open(f'lgbm_fold{fold}{add_suffix}.pkl', 'wb'))
        pred_i = model.predict(x_valid, num_iteration=model.best_iteration)
        oof_pred[x_valid.index] = pred_i
        score = round(RMSE(y_valid, pred_i), 5)
        print(f'Performance of the prediction: {score}')

    score = round(RMSE(y, oof_pred), 5)
    print(f'All Performance of the prediction: {score}')
    np.save(f'oof_lgbm{add_suffix}.npy', oof_pred)
    return oof_pred

def pred_lightgbm(X, data_dir: Path, add_suffix=''):
    models = glob(str(data_dir / f'lgbm*{add_suffix}.pkl'))
    models = [pickle.load(open(model, 'rb')) for model in models]
    preds = np.array([model.predict(X) for model in models])
    preds = np.mean(preds, axis=0)
    return preds

In [3]:
DATA_DIR = Path('../input')
SEED = 42
N_SPLITS = 5

train, test, clusters, feature_cols = read_data(DATA_DIR)
train_y = train['target']

train_X = processing_base(train, feature_cols)
test_X = processing_base(test, feature_cols)
train_X = pd.concat([train_X, processing_statistics(train)], axis=1)
test_X = pd.concat([test_X, processing_statistics(test)], axis=1)

oof_lgbm_final = np.zeros(len(train_X), dtype=np.float64)

SEEDS = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
for seed in SEEDS:
    folds = get_stratifiedkfold(train, N_SPLITS, seed)
    train_X['target'] = train['target'].values
    for col in ['ID', 'Location']:
        train_X, test_X = processing_targetencoding(train_X, test_X, col)
    train_X = train_X.drop('target', axis=1)

    params = {
        'objective': 'rmse',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'learning_rate': 0.1,
        'lambda_l1': 0.01,
        'lambda_l2': 0.01,
        'early_stopping_round': 100,
    }
    oof_lgbm = fit_lightgbm(train_X, train_y, params, folds, seed, f'_seed{seed}')
    oof_lgbm_final += oof_lgbm / len(SEEDS)
    train_X = train_X.drop(columns=['ID_target', 'Location_target'], axis=1)
print(round(RMSE(train_y, oof_lgbm_final), 5))

sub_lgbm_final = np.zeros(len(test_X), dtype=np.float64)
for seed in SEEDS:
    sub_lgbm = pred_lightgbm(test_X, Path(''), f'_seed{seed}')
    sub_lgbm_final += sub_lgbm / len(SEEDS)

np.save('submission.npy', sub_lgbm_final)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016587 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29906
[LightGBM] [Info] Number of data points in the train set: 74422, number of used features: 168
[LightGBM] [Info] Start training from score 50.692074
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 7.16755	valid_1's rmse: 7.38419
[200]	training's rmse: 6.45524	valid_1's rmse: 6.81838
[300]	training's rmse: 6.08459	valid_1's rmse: 6.59935
[400]	training's rmse: 5.8197	valid_1's rmse: 6.47137
[500]	training's rmse: 5.598	valid_1's rmse: 6.39202
[600]	training's rmse: 5.40683	valid_1's rmse: 6.32337
[700]	training's rmse: 5.24027	valid_1's rmse: 6.28482
[800]	training's rmse: 5.09238	valid_1's rmse: 6.24678
[900]	training's rmse: 4.94952	valid_1's rmse: 6.21592
[1000]	training's rmse: 4.8262	valid_1's rmse: 6.19885
[1100]	training's rmse: 4.71502	valid_1's r

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016056 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29908
[LightGBM] [Info] Number of data points in the train set: 74422, number of used features: 168
[LightGBM] [Info] Start training from score 50.662697
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 7.15929	valid_1's rmse: 7.43848
[200]	training's rmse: 6.44563	valid_1's rmse: 6.86976
[300]	training's rmse: 6.07094	valid_1's rmse: 6.64714
[400]	training's rmse: 5.79507	valid_1's rmse: 6.50977
[500]	training's rmse: 5.57328	valid_1's rmse: 6.42084
[600]	training's rmse: 5.38124	valid_1's rmse: 6.35814
[700]	training's rmse: 5.213	valid_1's rmse: 6.31171
[800]	training's rmse: 5.06495	valid_1's rmse: 6.27902
[900]	training's rmse: 4.9349	valid_1's rmse: 6.24925
[1000]	training's rmse: 4.80998	valid_1's rmse: 6.23025
[1100]	training's rmse: 4.69561	valid_1's 

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


...

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016669 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29901
[LightGBM] [Info] Number of data points in the train set: 74422, number of used features: 168
[LightGBM] [Info] Start training from score 50.644179
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 7.19758	valid_1's rmse: 7.35784
[200]	training's rmse: 6.4743	valid_1's rmse: 6.80986
[300]	training's rmse: 6.10107	valid_1's rmse: 6.5879
[400]	training's rmse: 5.8317	valid_1's rmse: 6.4594
[500]	training's rmse: 5.60188	valid_1's rmse: 6.36689
[600]	training's rmse: 5.41232	valid_1's rmse: 6.31075
[700]	training's rmse: 5.25056	valid_1's rmse: 6.26576
[800]	training's rmse: 5.09785	valid_1's rmse: 6.22863
[900]	training's rmse: 4.96049	valid_1's rmse: 6.19843
[1000]	training's rmse: 4.83663	valid_1's rmse: 6.17317
[1100]	training's rmse: 4.72517	valid_1's r

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016147 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29898
[LightGBM] [Info] Number of data points in the train set: 74423, number of used features: 168
[LightGBM] [Info] Start training from score 50.692633
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 7.16349	valid_1's rmse: 7.38965
[200]	training's rmse: 6.45244	valid_1's rmse: 6.80293
[300]	training's rmse: 6.08969	valid_1's rmse: 6.58444
[400]	training's rmse: 5.8362	valid_1's rmse: 6.46401
[500]	training's rmse: 5.61167	valid_1's rmse: 6.37325
[600]	training's rmse: 5.42599	valid_1's rmse: 6.30858
[700]	training's rmse: 5.26303	valid_1's rmse: 6.25845
[800]	training's rmse: 5.11419	valid_1's rmse: 6.22295
[900]	training's rmse: 4.98624	valid_1's rmse: 6.19969
[1000]	training's rmse: 4.85073	valid_1's rmse: 6.17305
[1100]	training's rmse: 4.73367	valid_1'

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018736 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 29910
[LightGBM] [Info] Number of data points in the train set: 74423, number of used features: 168
[LightGBM] [Info] Start training from score 50.644470
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 7.15376	valid_1's rmse: 7.38244
[200]	training's rmse: 6.44393	valid_1's rmse: 6.79618
[300]	training's rmse: 6.08216	valid_1's rmse: 6.57605
[400]	training's rmse: 5.81251	valid_1's rmse: 6.44526
[500]	training's rmse: 5.59566	valid_1's rmse: 6.36029
[600]	training's rmse: 5.40813	valid_1's rmse: 6.30423
[700]	training's rmse: 5.24299	valid_1's rmse: 6.26073
[800]	training's rmse: 5.09037	valid_1's rmse: 6.22306
[900]	training's rmse: 4.95262	valid_1's rmse: 6.19496
[1000]	training's rmse: 4.81896	valid_1's rmse: 6.1716
[1100]	training's rmse: 4.70703	valid_1'